---
title: "Lab 4 - Coffee Lovers Unite"
author: "Tyler Allen"
date: "2024-10-29"
format:
  html:
    cold-fold: true
    embed-resources: true
theme: "lux"
---

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:

Starbucks

Dunkin’ Donuts

In [20]:
#Starbucks
url = "https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [21]:
starbucks_table = soup.find('table')

rows_starbucks = []


for row in starbucks_table.find_all('tr'):
    cells = row.find_all(['th', 'td'])
    # Append
    rows_starbucks.append([cell.text.strip() for cell in cells])

starbucks_df = pd.DataFrame(rows_starbucks[1:], columns=rows_starbucks[0])
starbucks_df.head()

,State,Starbucks Stores 2023,Starbucks Stores 2021,Starbucks Stores 2024
0,California,"3,080","2,959","3,117"
1,Texas,"1,346","1,215","1,409"
2,Florida,844,786,892
3,Washington,741,739,736
4,New York,692,643,715


In [22]:
#Dunkin
url2 = "https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state"
response2 = requests.get(url2)
soup2 = BeautifulSoup(response2.text, 'html.parser')

In [23]:
dunkin_table = soup2.find('table')

rows_dunkin = []


for row in dunkin_table.find_all('tr'):
    cells = row.find_all(['th', 'td'])
    # Append
    rows_dunkin.append([cell.text.strip() for cell in cells])

dunkin_df = pd.DataFrame(rows_dunkin[1:], columns=rows_dunkin[0])
dunkin_df.head()

,State,Dunkin Locations 2024,Dunkin Locations 2023
0,New York,"1,431","1,414"
1,Massachusetts,"1,042","1,068"
2,Florida,909,883
3,New Jersey,872,866
4,Illinois,711,692


2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

In [24]:
import re

#melt Starbucks data
starbucks_long = pd.melt(starbucks_df, id_vars=['State'],
                         value_vars=['Starbucks Stores 2023', 'Starbucks Stores 2021', 'Starbucks Stores 2024'],
                         var_name='Year', value_name='Number of Stores')

# extract just the year from the year column
starbucks_long['Year'] = starbucks_long['Year'].apply(lambda x: re.search(r'\d{4}', x).group())

#add a column to identify starbucks
starbucks_long['Store Type'] = 'Starbucks'

#melt Dunkin data
dunkin_long = pd.melt(dunkin_df, id_vars=['State'],
                      value_vars=['Dunkin Locations 2024', 'Dunkin Locations 2023'],
                      var_name='Year', value_name='Number of Stores')

#extract only the year from the year column
dunkin_long['Year'] = dunkin_long['Year'].apply(lambda x: re.search(r'\d{4}', x).group())

#add a column to identify dunkin
dunkin_long['Store Type'] = 'Dunkin Donuts'

combined_long_df = pd.concat([starbucks_long, dunkin_long])

#replace non-digit characters (like commas) with empty strings using reg exp
combined_long_df['Number of Stores'] = combined_long_df['Number of Stores'].apply(lambda x: re.sub(r'\D', '', str(x)))

#convert number of stores to numeric
combined_long_df['Number of Stores'] = pd.to_numeric(combined_long_df['Number of Stores'], errors='coerce')
print(combined_long_df)

            State  Year  Number of Stores     Store Type
0      California  2023            3080.0      Starbucks
1           Texas  2023            1346.0      Starbucks
2         Florida  2023             844.0      Starbucks
3      Washington  2023             741.0      Starbucks
4        New York  2023             692.0      Starbucks
..            ...   ...               ...            ...
97        Montana  2023               0.0  Dunkin Donuts
98   North Dakota  2023               0.0  Dunkin Donuts
99         Oregon  2023               0.0  Dunkin Donuts
100  South Dakota  2023               0.0  Dunkin Donuts
101    Washington  2023              19.0  Dunkin Donuts

[255 rows x 4 columns]


In [25]:
#pivot the data with Year as columns and Number of Stores as the values
pivot_df = combined_long_df.pivot_table(index=['State', 'Store Type'], columns='Year', values='Number of Stores', fill_value=0)
#reset index
pivot_df = pivot_df.reset_index()
print(pivot_df)

Year          State     Store Type   2021   2023   2024
0           Alabama  Dunkin Donuts    0.0   59.0   69.0
1           Alabama      Starbucks   99.0   85.0    0.0
2            Alaska  Dunkin Donuts    0.0    0.0    0.0
3            Alaska      Starbucks   49.0   49.0    0.0
4           Arizona  Dunkin Donuts    0.0  102.0  110.0
..              ...            ...    ...    ...    ...
97    West Virginia      Starbucks   64.0   25.0    0.0
98        Wisconsin  Dunkin Donuts    0.0   83.0  100.0
99        Wisconsin      Starbucks  176.0  145.0    0.0
100         Wyoming  Dunkin Donuts    0.0    1.0    1.0
101         Wyoming      Starbucks   26.0   23.0    0.0

[102 rows x 5 columns]


4. Scrape the state names and populations from this wikipedia page. Merge these data with your coffee dataset.

In [26]:
#States
url3  = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population"
response3 = requests.get(url3)
soup3 = BeautifulSoup(response3.text, 'html.parser')

In [27]:
states_table = soup3.find('table')

rows_states = []


for row in states_table.find_all('tr'):
    cells = row.find_all(['th', 'td'])
    # Append
    rows_states.append([cell.text.strip() for cell in cells])

states_df = pd.DataFrame(rows_states[1:], columns=rows_states[0])
states_df.head()

,"Rank in states & territories, 2020","Rank in states & territories, 2010",State,"Census population, April 1, 2020[1][2]","Census population, April 1, 2010[1][2]","Percent change, 2010–2020[note 1]","Absolute change, 2010-2020","Total seats in the U.S. House of Representatives, 2023–2033",Census population per electoral vote[note 2],Census population per House seat,"Percent of the total U.S. population, 2020[note 3]"
0,1,1,California,"39,538,223","37,253,956",6.1%,"+2,284,267",52,"732,189","760,350",11.80%
1,2,2,Texas,"30,145,505","25,145,561",15.9%,"+3,999,944",38,"728,638","766,987",8.70%
2,3,4,Florida,"21,538,187","18,801,310",14.6%,"+2,736,877",28,"717,940","769,221",6.43%
3,4,3,New York,"20,201,249","19,378,102",4.2%,"+823,147",26,"721,473","776,971",6.03%
4,5,6,Pennsylvania,"13,002,700","12,702,379",2.4%,"+300,321",17,"684,353","764,865",3.88%


In [28]:
#test
print(states_df.columns)

Index(['Rank in states & territories, 2020',
       'Rank in states & territories, 2010', 'State',
       'Census population, April 1, 2020[1][2]',
       'Census population, April 1, 2010[1][2]',
       'Percent change, 2010–2020[note 1]', 'Absolute change, 2010-2020',
       'Total seats in the U.S. House of Representatives, 2023–2033',
       'Census population per electoral vote[note 2]',
       'Census population per House seat',
       'Percent of the total U.S. population, 2020[note 3]'],
      dtype='object')


In [29]:
states_df['State'] = states_df['State'].str.strip()
combined_long_df['State'] = combined_long_df['State'].str.strip()

#rename the population column
states_df.rename(columns={'Census population, April 1, 2020[1][2]': '2020 Population'}, inplace=True)

#merge
merged_df = pd.merge(combined_long_df, states_df[['State', '2020 Population']], on='State', how='left')
#cleaning population column
merged_df['2020 Population'] = merged_df['2020 Population'].str.replace(',', '').astype(float)
merged_df['2020 Population'].fillna(0, inplace=True)

print(merged_df.head())


        State  Year  Number of Stores Store Type  2020 Population
0  California  2023            3080.0  Starbucks       39538223.0
1       Texas  2023            1346.0  Starbucks       30145505.0
2     Florida  2023             844.0  Starbucks       21538187.0
3  Washington  2023             741.0  Starbucks        7705281.0
4    New York  2023             692.0  Starbucks       20201249.0


<ipython-input-29-5f2d70b15b3d>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['2020 Population'].fillna(0, inplace=True)


5. Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these). Merge these values into your big dataset. Note: these values may be repeated for each state.

The Starbucks stock prices I found are from the beginning of each year, found on YahooFinance. The Dunkin Donuts stock prices were estimated using ChatGPT using its market cap and revenue information.

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving Lab4FinanceInfo.xlsx to Lab4FinanceInfo (1).xlsx


In [16]:
financeinfo = pd.read_excel('Lab4FinanceInfo.xlsx', skiprows=1, header =1)
finance = financeinfo.dropna(axis=1, how='all')
finance

,Year,Store Type,Stock Price
0,2020,Starbucks,90.16
1,2020,Dunkin Donuts,38.51
2,2021,Starbucks,105.67
3,2021,Dunkin Donuts,44.91
4,2023,Starbucks,106.69
5,2023,Dunkin Donuts,45.34
6,2024,Starbucks,92.99
7,2024,Dunkin Donuts,39.52


In [37]:
# Convert Year columns to integers in both DataFrames
finance['Year'] = finance['Year'].astype(int)
merged_df['Year'] = merged_df['Year'].astype(int)

# Now merge the DataFrames
merge2 = pd.merge(finance, merged_df, on=['Store Type', 'Year'], how='left')

# Display the first few rows of the merged DataFrame
merge2

<ipython-input-37-a5d730587adc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance['Year'] = finance['Year'].astype(int)


,Year,Store Type,Stock Price,State,Number of Stores,2020 Population
0,2020,Starbucks,90.16,NaN,NaN,NaN
1,2020,Dunkin Donuts,38.51,NaN,NaN,NaN
2,2021,Starbucks,105.67,California,2959.0,39538223.0
3,2021,Starbucks,105.67,Texas,1215.0,30145505.0
4,2021,Starbucks,105.67,Florida,786.0,21538187.0
...,...,...,...,...,...,...
253,2024,Dunkin Donuts,39.52,Montana,0.0,1084225.0
254,2024,Dunkin Donuts,39.52,North Dakota,0.0,779094.0
255,2024,Dunkin Donuts,39.52,Oregon,0.0,4237256.0
256,2024,Dunkin Donuts,39.52,South Dakota,0.0,886667.0


6. Create a region variable in your dataset according to the scheme on this wikipedia page: Northeast, Midwest, South, West. You do not need to scrape this information.

In [44]:
#mapping each state to its respective region (used ChatGPT for time purposes)
state_to_region = {
    # Northeast
    'Connecticut': 'Northeast', 'Maine': 'Northeast', 'Massachusetts': 'Northeast',
    'New Hampshire': 'Northeast', 'Rhode Island': 'Northeast', 'Vermont': 'Northeast',
    'New Jersey': 'Northeast', 'New York': 'Northeast', 'Pennsylvania': 'Northeast',

    # Midwest
    'Indiana': 'Midwest', 'Illinois': 'Midwest', 'Michigan': 'Midwest', 'Ohio': 'Midwest',
    'Wisconsin': 'Midwest', 'Iowa': 'Midwest', 'Kansas': 'Midwest', 'Minnesota': 'Midwest',
    'Missouri': 'Midwest', 'Nebraska': 'Midwest', 'North Dakota': 'Midwest', 'South Dakota': 'Midwest',

    # South
    'Delaware': 'South', 'Florida': 'South', 'Georgia': 'South', 'Maryland': 'South',
    'North Carolina': 'South', 'South Carolina': 'South', 'Virginia': 'South', 'West Virginia': 'South',
    'Alabama': 'South', 'Kentucky': 'South', 'Mississippi': 'South', 'Tennessee': 'South',
    'Arkansas': 'South', 'Louisiana': 'South', 'Oklahoma': 'South', 'Texas': 'South',

    # West
    'Arizona': 'West', 'Colorado': 'West', 'Idaho': 'West', 'Montana': 'West',
    'Nevada': 'West', 'New Mexico': 'West', 'Utah': 'West', 'Wyoming': 'West',
    'Alaska': 'West', 'California': 'West', 'Hawaii': 'West', 'Oregon': 'West',
    'Washington': 'West'
}

#mapping the `State` column to the `Region` column
merge2['Region'] = merge2['State'].map(state_to_region)
merge2


,Year,Store Type,Stock Price,State,Number of Stores,2020 Population,Region
0,2020,Starbucks,90.16,NaN,NaN,NaN,NaN
1,2020,Dunkin Donuts,38.51,NaN,NaN,NaN,NaN
2,2021,Starbucks,105.67,California,2959.0,39538223.0,West
3,2021,Starbucks,105.67,Texas,1215.0,30145505.0,South
4,2021,Starbucks,105.67,Florida,786.0,21538187.0,South
...,...,...,...,...,...,...,...
253,2024,Dunkin Donuts,39.52,Montana,0.0,1084225.0,West
254,2024,Dunkin Donuts,39.52,North Dakota,0.0,779094.0,Midwest
255,2024,Dunkin Donuts,39.52,Oregon,0.0,4237256.0,West
256,2024,Dunkin Donuts,39.52,South Dakota,0.0,886667.0,Midwest


7. Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.


Starbucks and Dunkin Donuts have different prevalance across regions. For example, Dunkin Donuts is more concentrated in the Northeast, while Starbucks is more prevalent in the West and South. Furthermore, states like California and Texas have a significantly higher number of Starbucks locations, however Dunkin Donuts is more densely located in states like New York and New Jersey despite having fewer locations.  

How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.


I don't know how the stock prices of Starbucks and Dunkin Donuts changes by state and region with the information I have, however I can infer that states with more locations, such as California, could contribute to higher revenue and in turn raise the stock price. Also, having high store counts in higher-populated states such as California or Texas probably lead to higher revenue compared to having those locations in less-populated states such as Wyoming.

Does the distribution of each chain’s stores match population distribution, by both state/region?



I think for the most part, the number of stores matches the population distribution by state and region. Some of the highest populated states (California, Florida, Texas) have more stores in general. However, it seems that Dunkin Donuts prioritizes the East coast even though those states don't have as big of a population size as other states.

**Automation**

Convert your code for Exercises 1-3 above to a function that takes a single argument: the URL. This function should

Scrape the information on state names and corresponding number of store locations on the webpage specified (assume the page has a table in the same form and placement as the ones you scraped above)

Extract the name of the company from either the URL specified or the webpage (assume the URL will have the same format as the ones used above)

Return a clean, organized and tidy dataset. Find a page other than Starbucks and Dunkin’ Donuts to test this on to confirm that it works. It’s fine if this is not related to coffee.

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

#function to scrape data from a given URL
def scrape_store_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    #extracting company name from the URL using regular expressions
    company_name = re.search(r'(?<=state-rankings/)(.*?)(?=-by-state)', url)
    company_name = company_name.group(1).replace('-', ' ').title() if company_name else 'Unknown'

    table = soup.find('table')

    rows = []

    #to extract the data in the table
    for row in table.find_all('tr'):
        cells = row.find_all(['th', 'td'])
        rows.append([cell.text.strip() for cell in cells])

    #convert to df and assign column names
    df = pd.DataFrame(rows[1:], columns=rows[0])

    #add a column to show which company it is
    df['Company'] = company_name

    #cleaning data
    if 'Number of Stores' in df.columns:
        df['Number of Stores'] = df['Number of Stores'].str.replace(',', '').astype(int)

    return df


In [56]:
#Walmart Test
url = "https://worldpopulationreview.com/state-rankings/walmart-stores-by-state"
walmart_df = scrape_store_data(url)
walmart_df.head()

,State,# of Stores,Company
0,Texas,517,Walmart Stores
1,Florida,341,Walmart Stores
2,California,280,Walmart Stores
3,North Carolina,192,Walmart Stores
4,Georgia,189,Walmart Stores
